In [4]:
import pandas as pd 

```
select  c.table_name, c.column_name, c.data_type, c.*
from information_schema.columns c
where 1=1
and c.table_catalog = 'faculty-Cornell-CS'
and c.table_schema = 'main'
and c.table_name in (
	--'t_note'
	select table_name from information_schema.tables t
	where t.table_catalog = 'faculty-Cornell-CS'
	and t.table_schema = 'main'
)
order by c.table_catalog,c.table_schema,c.table_name, c.ordinal_position
```

In [5]:
df = pd.read_csv("columns_202304020735.csv")

In [7]:
df = df[["table_name","column_name"]]

In [12]:
df_list= df.groupby('table_name')['column_name'].apply(list)

In [15]:
TABLE_COLUMNS = df_list.to_dict()

In [16]:
TABLE_COLUMNS

{'t_faculty': ['name',
  'job_title',
  'phd_univ',
  'phd_year',
  'research_area',
  'research_concentration',
  'research_focus',
  'url',
  'img_url',
  'phone',
  'email',
  'cell_phone',
  'office_address',
  'department',
  'school'],
 't_note': ['title', 'note', 'url', 'tags', 'ts', 'id'],
 't_research_group': ['research_group', 'url']}

In [1]:
s = """
title
note
url
tags
ts
id
"""

In [2]:
cols = [i.strip() for i in s.split("\n") if i.strip()]

In [3]:
cols

['title', 'note', 'url', 'tags', 'ts', 'id']

In [17]:
'title'.capitalize()

'Title'

In [7]:
def _gen_label(col):
    "Convert table column into form label"
    if "_" not in col:
        if col.upper() in ["URL","ID"]:
            return col.upper()
        elif col.upper() == "TS":
            return "Timestamp"
        return col.capitalize()

    cols = []
    for c in col.split("_"):
        c  = c.strip()
        if not c: continue
        cols.append(c.capitalize())
    return " ".join(cols)

In [ ]:
txt = """
	is_system_col: T/F,   # read-only, maybe hidden such as id,ts,uid
	is_user_key: T/F,
	is_required: T/F,
	is_visible: T/F,  # appear in form or not
	is_editable: T/F,
	is_clickable: T/F,  # URL link

	form_column: left-n (default, required) | mid-n | right-n  # n sequence, mid/right
	widget_type: text_input (default) | selectbox | text_area | ... (see st API docs)
	label_text: "C1", # optional, use _gen_label(col) when unavailable
"""

In [3]:
props = [l.strip().split(":")[0] for l in txt.split("\n") if l.strip()]

In [4]:
props

['is_system_col',
 'is_user_key',
 'is_required',
 'is_visible',
 'is_editable',
 'is_clickable',
 'form_column',
 'widget_type',
 'label_text']

In [8]:
col_def = {}
for p in props:
    if "is_" in p:
        col_def.update({p: False})
    elif p == 'form_column':
        col_def.update({p: "left-1"})
    elif p == 'widget_type':
        col_def.update({p: "text_input"})
    else:
        col_def.update({p: _gen_label(p)})

In [9]:
col_def

{'is_system_col': False,
 'is_user_key': False,
 'is_required': False,
 'is_visible': False,
 'is_editable': False,
 'is_clickable': False,
 'form_column': 'left-1',
 'widget_type': 'text_input',
 'label_text': 'Label Text'}

In [11]:
COL_PROPS = {
    "t_note" : {
        "title": {
            'is_system_col': False,
            'is_user_key': True,
            'is_required': True,
            'is_visible': True,
            'is_editable': True,
            'is_clickable': False,
            'form_column': 'col1-1',
            'widget_type': 'text_input',
            'label_text': 'Title'  
        },
        "url": {
            'is_system_col': False,
            'is_user_key': False,
            'is_required': False,
            'is_visible': True,
            'is_editable': True,
            'is_clickable': True,
            'form_column': 'col1-2',
            'widget_type': 'text_input',
            'label_text': 'URL'  
        },
        "tags": {
            'is_system_col': False,
            'is_user_key': False,
            'is_required': False,
            'is_visible': True,
            'is_editable': True,
            'is_clickable': False,
            'form_column': 'col1-3',
            'widget_type': 'text_input',
        },
        "id": {
            'is_system_col': True,
            'is_user_key': False,
            'is_required': True,
            'is_visible': True,
            'is_editable': False,
            'is_clickable': False,
            'form_column': 'col2-1',
            'widget_type': 'text_input',
            'label_text': 'ID'                  
        },
        "note": {
            'is_system_col': False,
            'is_user_key': False,
            'is_required': False,
            'is_visible': True,
            'is_editable': True,
            'is_clickable': False,
            'form_column': 'col2-2',
            'widget_type': 'text_area',
        },
        "ts": {
            'is_system_col': True,
            'is_user_key': False,
            'is_required': False,
            'is_visible': False,
            'is_editable': False,
            'is_clickable': False,
        },
        "ref_type": {
            'is_system_col': False,
            'is_user_key': False,
            'is_required': False,
            'is_visible': False,
            'is_editable': False,
            'is_clickable': False,
        },
        "ref_key": {
            'is_system_col': False,
            'is_user_key': False,
            'is_required': False,
            'is_visible': False,
            'is_editable': False,
            'is_clickable': False,
        },
    }
}


In [3]:
def _get_columns(table_name, prop_name="is_visible"):
    return [k for k,v in COL_PROPS[table_name].items() if v.get(prop_name, False) ]

In [4]:
_get_columns("t_note", prop_name="is_system_col")

['id', 'ts']

In [5]:
_get_columns("t_note", prop_name="is_user_key")

['title']

In [6]:
visible_columns = _get_columns("t_note")
visible_columns

['title', 'url', 'tags', 'id', 'note']

In [7]:
_get_columns("t_note", prop_name="is_required")

['title', 'id']

In [8]:
_get_columns("t_note", prop_name="is_editable")

['title', 'url', 'tags', 'note']

In [9]:
_get_columns("t_note", prop_name="is_clickable")

['url']

In [13]:
table_name = "t_note"
COL_DEFS = COL_PROPS[table_name]

col1_columns = []
for c in visible_columns:
    if COL_DEFS[c].get("form_column", "").startswith("col1-"):
        col1_columns.append(c)
        
col1_columns

['title', 'url', 'tags']

In [14]:
col2_columns = []
for c in visible_columns:
    if COL_DEFS[c].get("form_column", "").startswith("col2-"):
        col2_columns.append(c)
        
col2_columns

['id', 'note']